In [7]:

from neo4j import GraphDatabase
import json

# Connect to Neo4j
uri = "neo4j+s://626ee68a.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j", "sYKDgXkyjO_zR3b3UMEoq9MqzEoyP5p2c7BFtoH-6i0"))

def create_nodes_and_edges(data):
    with driver.session() as session:
        # Create nodes
        for node in data['nodes']:
            session.run(
                "CREATE (:Station {id: $id, name: $name, latitude: $lat, longitude: $lon})",
                id=node['id'], name=node['text'], lat=node['latitude'], lon=node['longitude']
            )
        # Add edges here if defined in your data
        for edge in data['edges']:
            session.run(
                """
                MATCH (a:Station {id: $source}), (b:Station {id: $target})
                MERGE (a)-[:CONNECTS {line: $line, color: $color}]->(b)
                """,
                source=edge['source'],
                target=edge['target'],
                line=edge['line'],
                color=edge['color']
            )
# Load the JSON
with open("metro.json", "r") as file:
    metro_data = json.load(file)

create_nodes_and_edges(metro_data)
driver.close()
